In [1]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
import statistics

%matplotlib inline

In [44]:
alpha = pd.read_csv('datasets/data_alpha.csv')

In [45]:
alpha

,Data,ABEV3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BBSE3,BEEF3,BPAC11,...,TAEE11,TIMP3,TOTS3,UGPA3,USIM5,VALE3,VIVT4,VVAR3,WEGE3,YDUQ3
0,12/06/2020,13.2200,21.22,48.87,32.9200,19.5600,21.40,25.97,12.71,61.34,...,28.52,13.88,20.87,17.70,6.53,52.4900,48.4100,14.76,43.62,33.17
1,10/06/2020,13.9600,23.50,50.74,34.2100,20.3600,21.88,26.85,12.77,63.36,...,29.05,14.30,21.49,18.62,6.89,54.2000,49.5900,14.95,44.99,34.90
2,09/06/2020,14.3200,25.78,51.07,35.7900,21.2100,22.95,27.62,12.59,65.58,...,28.81,14.48,21.11,19.27,7.30,54.9900,49.0000,14.97,45.81,36.64
3,08/06/2020,14.6000,27.35,51.72,36.3900,21.4300,23.47,28.40,13.05,66.02,...,28.94,14.44,21.03,19.13,7.45,54.7800,48.5400,15.50,45.80,37.15
4,05/06/2020,14.0500,21.16,49.07,35.2000,20.4500,22.30,28.49,13.01,60.24,...,28.93,14.60,20.76,18.68,6.90,54.6100,47.7700,14.85,44.33,38.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4964,16/06/2000,7.7185,NaN,NaN,2.0367,2.6572,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7500,26.0936,NaN,NaN,NaN
4965,15/06/2000,7.7552,NaN,NaN,2.1200,2.5243,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7256,26.4651,NaN,NaN,NaN
4966,14/06/2000,7.7414,NaN,NaN,2.1467,2.5022,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7306,26.8365,NaN,NaN,NaN
4967,13/06/2000,7.7476,NaN,NaN,2.1000,2.4937,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.7400,26.2794,NaN,NaN,NaN


In [52]:
for idx in reversed(alpha.index):
    print(idx, alpha.loc[idx, 'Data'], alpha.loc[idx, 'Data'])


4968 12/06/2000 12/06/2000
4967 13/06/2000 13/06/2000
4966 14/06/2000 14/06/2000
4965 15/06/2000 15/06/2000
4964 16/06/2000 16/06/2000
4963 19/06/2000 19/06/2000
4962 20/06/2000 20/06/2000
4961 21/06/2000 21/06/2000
4960 22/06/2000 22/06/2000
4959 23/06/2000 23/06/2000
4958 26/06/2000 26/06/2000
4957 27/06/2000 27/06/2000
4956 28/06/2000 28/06/2000
4955 29/06/2000 29/06/2000
4954 30/06/2000 30/06/2000
4953 03/07/2000 03/07/2000
4952 04/07/2000 04/07/2000
4951 05/07/2000 05/07/2000
4950 06/07/2000 06/07/2000
4949 07/07/2000 07/07/2000
4948 10/07/2000 10/07/2000
4947 11/07/2000 11/07/2000
4946 12/07/2000 12/07/2000
4945 13/07/2000 13/07/2000
4944 14/07/2000 14/07/2000
4943 17/07/2000 17/07/2000
4942 18/07/2000 18/07/2000
4941 19/07/2000 19/07/2000
4940 20/07/2000 20/07/2000
4939 21/07/2000 21/07/2000
4938 24/07/2000 24/07/2000
4937 25/07/2000 25/07/2000
4936 26/07/2000 26/07/2000
4935 27/07/2000 27/07/2000
4934 28/07/2000 28/07/2000
4933 31/07/2000 31/07/2000
4932 01/08/2000 01/08/2000
4

4042 26/02/2004 26/02/2004
4041 27/02/2004 27/02/2004
4040 01/03/2004 01/03/2004
4039 02/03/2004 02/03/2004
4038 03/03/2004 03/03/2004
4037 04/03/2004 04/03/2004
4036 05/03/2004 05/03/2004
4035 08/03/2004 08/03/2004
4034 09/03/2004 09/03/2004
4033 10/03/2004 10/03/2004
4032 11/03/2004 11/03/2004
4031 12/03/2004 12/03/2004
4030 15/03/2004 15/03/2004
4029 16/03/2004 16/03/2004
4028 17/03/2004 17/03/2004
4027 18/03/2004 18/03/2004
4026 19/03/2004 19/03/2004
4025 22/03/2004 22/03/2004
4024 23/03/2004 23/03/2004
4023 24/03/2004 24/03/2004
4022 25/03/2004 25/03/2004
4021 26/03/2004 26/03/2004
4020 29/03/2004 29/03/2004
4019 30/03/2004 30/03/2004
4018 31/03/2004 31/03/2004
4017 01/04/2004 01/04/2004
4016 02/04/2004 02/04/2004
4015 05/04/2004 05/04/2004
4014 06/04/2004 06/04/2004
4013 07/04/2004 07/04/2004
4012 08/04/2004 08/04/2004
4011 12/04/2004 12/04/2004
4010 13/04/2004 13/04/2004
4009 14/04/2004 14/04/2004
4008 15/04/2004 15/04/2004
4007 16/04/2004 16/04/2004
4006 19/04/2004 19/04/2004
4

3108 09/11/2007 09/11/2007
3107 12/11/2007 12/11/2007
3106 13/11/2007 13/11/2007
3105 14/11/2007 14/11/2007
3104 16/11/2007 16/11/2007
3103 19/11/2007 19/11/2007
3102 21/11/2007 21/11/2007
3101 22/11/2007 22/11/2007
3100 23/11/2007 23/11/2007
3099 26/11/2007 26/11/2007
3098 27/11/2007 27/11/2007
3097 28/11/2007 28/11/2007
3096 29/11/2007 29/11/2007
3095 30/11/2007 30/11/2007
3094 03/12/2007 03/12/2007
3093 04/12/2007 04/12/2007
3092 05/12/2007 05/12/2007
3091 06/12/2007 06/12/2007
3090 07/12/2007 07/12/2007
3089 10/12/2007 10/12/2007
3088 11/12/2007 11/12/2007
3087 12/12/2007 12/12/2007
3086 13/12/2007 13/12/2007
3085 14/12/2007 14/12/2007
3084 17/12/2007 17/12/2007
3083 18/12/2007 18/12/2007
3082 19/12/2007 19/12/2007
3081 20/12/2007 20/12/2007
3080 21/12/2007 21/12/2007
3079 26/12/2007 26/12/2007
3078 27/12/2007 27/12/2007
3077 28/12/2007 28/12/2007
3076 02/01/2008 02/01/2008
3075 03/01/2008 03/01/2008
3074 04/01/2008 04/01/2008
3073 07/01/2008 07/01/2008
3072 08/01/2008 08/01/2008
3

2108 07/12/2011 07/12/2011
2107 08/12/2011 08/12/2011
2106 09/12/2011 09/12/2011
2105 12/12/2011 12/12/2011
2104 13/12/2011 13/12/2011
2103 14/12/2011 14/12/2011
2102 15/12/2011 15/12/2011
2101 16/12/2011 16/12/2011
2100 19/12/2011 19/12/2011
2099 20/12/2011 20/12/2011
2098 21/12/2011 21/12/2011
2097 22/12/2011 22/12/2011
2096 23/12/2011 23/12/2011
2095 26/12/2011 26/12/2011
2094 27/12/2011 27/12/2011
2093 28/12/2011 28/12/2011
2092 29/12/2011 29/12/2011
2091 02/01/2012 02/01/2012
2090 03/01/2012 03/01/2012
2089 04/01/2012 04/01/2012
2088 05/01/2012 05/01/2012
2087 06/01/2012 06/01/2012
2086 09/01/2012 09/01/2012
2085 10/01/2012 10/01/2012
2084 11/01/2012 11/01/2012
2083 12/01/2012 12/01/2012
2082 13/01/2012 13/01/2012
2081 16/01/2012 16/01/2012
2080 17/01/2012 17/01/2012
2079 18/01/2012 18/01/2012
2078 19/01/2012 19/01/2012
2077 20/01/2012 20/01/2012
2076 23/01/2012 23/01/2012
2075 24/01/2012 24/01/2012
2074 26/01/2012 26/01/2012
2073 27/01/2012 27/01/2012
2072 30/01/2012 30/01/2012
2

1108 23/12/2015 23/12/2015
1107 28/12/2015 28/12/2015
1106 29/12/2015 29/12/2015
1105 30/12/2015 30/12/2015
1104 04/01/2016 04/01/2016
1103 05/01/2016 05/01/2016
1102 06/01/2016 06/01/2016
1101 07/01/2016 07/01/2016
1100 08/01/2016 08/01/2016
1099 11/01/2016 11/01/2016
1098 12/01/2016 12/01/2016
1097 13/01/2016 13/01/2016
1096 14/01/2016 14/01/2016
1095 15/01/2016 15/01/2016
1094 18/01/2016 18/01/2016
1093 19/01/2016 19/01/2016
1092 20/01/2016 20/01/2016
1091 21/01/2016 21/01/2016
1090 22/01/2016 22/01/2016
1089 26/01/2016 26/01/2016
1088 27/01/2016 27/01/2016
1087 28/01/2016 28/01/2016
1086 29/01/2016 29/01/2016
1085 01/02/2016 01/02/2016
1084 02/02/2016 02/02/2016
1083 03/02/2016 03/02/2016
1082 04/02/2016 04/02/2016
1081 05/02/2016 05/02/2016
1080 10/02/2016 10/02/2016
1079 11/02/2016 11/02/2016
1078 12/02/2016 12/02/2016
1077 15/02/2016 15/02/2016
1076 16/02/2016 16/02/2016
1075 17/02/2016 17/02/2016
1074 18/02/2016 18/02/2016
1073 19/02/2016 19/02/2016
1072 22/02/2016 22/02/2016
1

109 02/01/2020 02/01/2020
108 03/01/2020 03/01/2020
107 06/01/2020 06/01/2020
106 07/01/2020 07/01/2020
105 08/01/2020 08/01/2020
104 09/01/2020 09/01/2020
103 10/01/2020 10/01/2020
102 13/01/2020 13/01/2020
101 14/01/2020 14/01/2020
100 15/01/2020 15/01/2020
99 16/01/2020 16/01/2020
98 17/01/2020 17/01/2020
97 20/01/2020 20/01/2020
96 21/01/2020 21/01/2020
95 22/01/2020 22/01/2020
94 23/01/2020 23/01/2020
93 24/01/2020 24/01/2020
92 27/01/2020 27/01/2020
91 28/01/2020 28/01/2020
90 29/01/2020 29/01/2020
89 30/01/2020 30/01/2020
88 31/01/2020 31/01/2020
87 03/02/2020 03/02/2020
86 04/02/2020 04/02/2020
85 05/02/2020 05/02/2020
84 06/02/2020 06/02/2020
83 07/02/2020 07/02/2020
82 10/02/2020 10/02/2020
81 11/02/2020 11/02/2020
80 12/02/2020 12/02/2020
79 13/02/2020 13/02/2020
78 14/02/2020 14/02/2020
77 17/02/2020 17/02/2020
76 18/02/2020 18/02/2020
75 19/02/2020 19/02/2020
74 20/02/2020 20/02/2020
73 21/02/2020 21/02/2020
72 27/02/2020 27/02/2020
71 28/02/2020 28/02/2020
70 02/03/2020 0